# WSmart+ Route Optimization

In [1]:
from notebook_setup import setup_home_directory, setup_google_colab


NOTEBOOK_NAME = 'optimization'
home_dir = setup_home_directory(NOTEBOOK_NAME)
IN_COLAB, gdrive, gfiles = setup_google_colab(NOTEBOOK_NAME)

Setup completed - added home_dir to system path: /home/pkhunter/Repositories/WSmart-Route


In [2]:
if IN_COLAB:
    %pip install fast-tsp
    %pip install gurobipy
    %pip install shapely
    %pip install matplotlib
    %pip install tqdm
    %pip install pandas
    %pip install torch
    %pip install cuda-cudart
    %pip install cudatoolkit
    %pip install jupyter
    %pip install networkx
    %pip install numpy
    %pip install torch_geometric
    %pip install ortools

In [3]:
import re
import os
import sys
import time
import torch
import traceback
import numpy as np
import pandas as pd
import pprint as pp
import gurobipy as gp

%matplotlib inline
import matplotlib.pyplot as plt

from tqdm import tqdm
from __future__ import print_function
from logic.src.utils.definitions import SIM_METRICS, TQDM_COLOURS, DAY_METRICS
from logic.src.utils.functions import load_model
from logic.src.utils.plot_utils import plot_attention_maps_wrapper
from logic.src.utils.log_utils import log_to_json, log_plot, log_to_pickle
from logic.src.utils.setup_utils import setup_model, setup_env, setup_hrl_manager
from logic.src.pipeline.simulator.bins import Bins
from logic.src.pipeline.simulator.wsmart_bin_analysis import OldGridBase
from logic.src.pipeline.simulator.day import set_daily_waste, get_daily_results
from logic.src.pipeline.simulator.processor import process_data, process_model_data
from logic.src.pipeline.simulator.network import compute_distance_matrix, get_paths_between_states, apply_edges
from logic.src.pipeline.simulator.loader import load_indices, load_depot, load_simulator_data, load_area_and_waste_type_params
from logic.src.or_policies import (
    get_route_cost, find_route,
    create_points, find_solutions,
    policy_regular, policy_gurobi_vrpp,
    policy_last_minute, policy_last_minute_and_path,
    policy_lookahead, policy_lookahead_vrpp, policy_lookahead_sans,
    policy_hexaly_vrpp
)


SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

FIXED_POINT_NOTATION = True # False to use scientific notation instead
FLOAT_DIGITS_PRECISION = 15
np.set_printoptions(precision=FLOAT_DIGITS_PRECISION)
np.set_printoptions(suppress=FIXED_POINT_NOTATION)

pd.options.mode.chained_assignment = None  # default='warn'
if IN_COLAB: 
    gdrive.mount('/content/drive')

# Required to use matplotlib in Windows without breaking the Kernel
if os.name == 'nt':
    os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

## Simulator

### Experimental Setup

In [4]:
Ndays = 30
number_of_bins = 100
n_bins = number_of_bins + 1 #with depot
NC = n_bins
binsids = np.arange(0, NC-1).tolist()

area = "Rio Maior"
waste_type = 'plastic'
data_distribution = "gamma" #"emp"
area = re.sub(r'[^a-zA-Z]', '', area.lower()) #area.translate(str.maketrans('', '', '-_ ')).lower()

model_id = 0
gamma_option = 0
model_names = ["amgat_hrl"] #["amgat", "amgac", "amtgc"]
inner_dir = f"gamma{gamma_option+1}" if data_distribution == 'gamma' else "emp"

data_dir = os.path.join(home_dir, "data", "wsr_simulator")
output_dir = os.path.join(home_dir, "assets", "output", f"{Ndays}_days", f"{area}_{number_of_bins}")
try:
    os.makedirs(data_dir, exist_ok=True)
    os.makedirs(output_dir, exist_ok=True)
except Exception:
    traceback.print_exc(file=sys.stdout)

In [5]:
models = {}
configs = {}
problem = "cwcvrp"
decode_type = "greedy"
softmax_temperature = 1
for model_name in model_names:
    model_path = os.path.join(home_dir, "assets", "model_weights", f"{problem}{number_of_bins}_{area}_{waste_type}", inner_dir, model_name)
    try:
        model, config = load_model(model_path)
        pp.pprint(config)
        
        device = torch.device("cpu" if not torch.cuda.is_available() else f"cuda:{torch.cuda.device_count()-1}")
        print("Device set to", device)

        hrl_manager = setup_hrl_manager({'model_path': model_path}, device, config)
        if hrl_manager is not None: print("Loaded HRL Manager...")

        models[model_name] = model
        configs[model_name] = config
        models[model_name].to(device)
        models[model_name].eval()
        models[model_name].set_decode_type(decode_type, temp=softmax_temperature)
    except Exception as e:
        print(f"Failed to load {model_name} model from {model_path}")
        traceback.print_exc(file=sys.stdout)

  [*] Loading model from /home/pkhunter/Repositories/WSmart-Route/assets/model_weights/cwcvrp100_riomaior_plastic/gamma1/amgat_hrl/epoch-30.pt
{'accumulation_steps': 1,
 'activation': 'gelu',
 'af_nparams': 3,
 'af_param': 1.0,
 'af_replacement': None,
 'af_threshold': None,
 'af_urange': [0.125, 0.3333333333333333],
 'aggregation': 'sum',
 'aggregation_graph': 'mean',
 'area': 'riomaior',
 'baseline': None,
 'batch_size': 256,
 'bl_alpha': 0.05,
 'bl_warmup_epochs': 0,
 'cb_context_features': ['waste', 'overflow', 'length', 'visited_ratio', 'day'],
 'cb_epsilon_decay': 0.995,
 'cb_exploration_method': 'ucb',
 'cb_features_aggregation': 'avg',
 'cb_min_epsilon': 0.01,
 'cb_num_configs': 10,
 'checkpoint_encoder': False,
 'checkpoint_epochs': 1,
 'checkpoints_dir': 'model_weights',
 'data_distribution': 'gamma1',
 'device': 'cuda:0',
 'distance_method': 'gmaps',
 'dm_filepath': 'data/wsr_simulator/distance_matrix/gmaps_distmat_plastic[riomaior].csv',
 'dropout': 0.1,
 'edge_method': 'kn

In [6]:
last_minute_cfs = []
last_minute_variants = 'both' # 'only'|'path'|'both'

regular_levels = []

look_ahead_configs = []
look_ahead_configurations = {
    'a': [500,75,0.7,0,0.095,0,0], 
    'b': [2000,75,0.7,0,0.095,0,0]
}
look_ahead_variants = [] # 'base', 'vrpp', 'sans'

gp_params = []
if gp_params or 'vrpp' in look_ahead_variants:
    gp_env_params = {'OutputFlag': 0}
    gp_env = gp.Env(params=gp_env_params)

hex_params = []

In [7]:
policies = []
if len(model_names) > 0:
    for model_name in model_names:
        policy = "{}{}_{}".format(model_name, f"{model_id}" if model_id >= 1 else "", inner_dir)
        policies.append(policy)

if last_minute_cfs:
    for lmcf in last_minute_cfs:
        if last_minute_variants in ['only', 'both']:
            policy = f"policy_last_minute{lmcf}_gamma{gamma_option+1}" if inner_dir[:-1] == "gamma" else f"policy_last_minute{lmcf}_emp"
            policies.append(policy)
        if last_minute_variants in ['path', 'both']:
            policy = f"policy_last_minute_and_path{lmcf}_gamma{gamma_option+1}" if inner_dir[:-1] == "gamma" else f"policy_last_minute_and_path{lmcf}_emp"
            policies.append(policy)

if regular_levels:
    for lvl in regular_levels:
        policy = f"policy_regular{lvl}_gamma{gamma_option+1}" if inner_dir[:-1] == "gamma" else f"policy_regular{lvl}_emp"
        policies.append(policy)

for lac in look_ahead_configs:
    for lav in look_ahead_variants:
        if lav == 'base':
            policy = f"policy_look_ahead_{lac}_gamma{gamma_option+1}" if inner_dir[:-1] == "gamma" else f"policy_look_ahead_{lac}_emp"
        else:
            policy = f"policy_look_ahead_{lac}_{lav}_gamma{gamma_option+1}" if inner_dir[:-1] == "gamma" else f"policy_look_ahead_{lac}_{lav}_emp"
        policies.append(policy)

if gp_params:
    for gpp in gp_params:
        policy = f"gurobi_vrpp{gpp}_gamma{gamma_option+1}" if inner_dir[:-1] == "gamma" else f"gurobi_vrpp{gpp}_emp"
        policies.append(policy)

if hex_params:
    for hexp in hex_params:
        policy = f"hexaly_vrpp{hexp}_gamma{gamma_option+1}" if inner_dir[:-1] == "gamma" else f"hexaly_vrpp{hexp}_emp"
        policies.append(policy)

#policies = policies[1:] #+ policies[:1]
print(policies)

['amgat_hrl_gamma1']


In [8]:
depot = load_depot(data_dir, area)
data, bins_coordinates = load_simulator_data(data_dir, number_of_bins, area, waste_type)
assert data.shape == bins_coordinates.shape

print(f"Area {area} ({bins_coordinates.shape[0]} full) for {number_of_bins} bins")
print(f"Lat: [{bins_coordinates['Lat'].min()}, {bins_coordinates['Lat'].max()}]")
print(f"Lng: [{bins_coordinates['Lng'].min()}, {bins_coordinates['Lng'].max()}]")

edge_thresh = 1.0
edge_method = "knn"
norm_method = "mmn"
dist_mat_method = "gmaps"
if area == 'riomaior':
    grid = OldGridBase(data_dir, area)
else:
    grid = None

Area riomaior (173 full) for 100 bins
Lat: [39.25353454, 39.4429361111111]
Lng: [-8.984290944, -8.79266007]


In [9]:
depot_tmp = depot.copy()
depot_location = "og"
if depot_location == 'mean':
    depot_tmp['Lat'] = bins_coordinates['Lat'].mean()
    depot_tmp['Lng'] = bins_coordinates['Lng'].mean()
else:
    assert depot_location == 'og'

### Run Experiments on Simulator

In [10]:
Nsamples = 10
start_id = 0
assert start_id < Nsamples

log_filepath = os.path.join(output_dir, f'log_mean_{Nsamples}N.json')
if Nsamples > 1:
    logstd_filepath = os.path.join(output_dir, f'log_std_{Nsamples}N.json')
    logfull_filepath = os.path.join(output_dir, f"log_full_{Nsamples}N.json")

data_size = bins_coordinates.shape[0]
dm_filepath = os.path.join(data_dir, "distance_matrix", f"gmaps_distmat_{waste_type}[{area}].csv")
daily_log_path = os.path.join(output_dir, f"daily_{inner_dir}_{Nsamples}N.json")
if data_size > number_of_bins:
    idx_filename = f"graphs_{number_of_bins}V_1N_{waste_type}.json"
    indices_ls = load_indices(idx_filename, Nsamples, number_of_bins, data_size)
else:
    indices_ls = [None] * Nsamples

daily_waste_path = os.path.join(data_dir, "daily_waste", "{}{}_{}_wsr31_N10_seed{}.pkl".format(area, number_of_bins, inner_dir, SEED))
if not os.path.exists(daily_waste_path): 
    load_waste = False
    daily_waste_path = None
    print(f"Specified daily waste fill file {daily_waste_path} does not exist")
else:
    load_waste = True

In [14]:
log = []
if Nsamples > 1:
    tmp_log = {pol: [] for pol in policies}

run_tsp = False
regular_cache = True
full_daily_log = {}
attention_dict = {model_name: [] for model_name in model_names}
for sample_id in range(start_id, Nsamples):
    indices = indices_ls[sample_id]
    new_data, coordinates = process_data(data, bins_coordinates, depot_tmp, indices=indices)
    distance_matrix = compute_distance_matrix(coordinates, dist_mat_method, focus_idx=indices, dm_filepath=dm_filepath)
    dist_matrix_edges, shortest_paths, adj_matrix = apply_edges(distance_matrix, edge_thresh, edge_method)
    pathbetweenstates = get_paths_between_states(n_bins, shortest_paths)
    distancesC = np.round(dist_matrix_edges*10).astype('int32')
    if len(models) > 0: distC_tensor = torch.from_numpy(distancesC).to(device)
    tour_ls = []
    cost_ls = []
    for pol_id, pol in enumerate(policies):
        desc = f"{pol} #{sample_id}"
        policy = pol.rsplit('_', 1)[0]
        daily_log = {key: [] for key in DAY_METRICS}
        if len(models) > 0:
            model_strip_name = re.split(r'[^a-zA-Z]', pol, maxsplit=1)[0]
            if model_strip_name in ["amgat", "amgac", "amtgc"]:
                model = models[policy]
                config = configs[policy]
                model_data, graph, profit_vars = process_model_data(coordinates, distancesC, device, norm_method, config, 
                                                                edge_thresh, edge_method, area, waste_type, adj_matrix)
        else:
            model_data, graph = (None, None)

        overflows = 0
        cached = [] if regular_cache else None
        current_collection_day = 0
        bins = Bins(NC - 1, data_dir, data_distribution, grid, waste_file=daily_waste_path, waste_type=waste_type, area=area)
        bins.set_indices(indices)
        if data_distribution == 'gamma':
            bins.setGammaDistribution(option=gamma_option)
        if daily_waste_path is not None:
            bins.set_sample_waste(sample_id)

        colour = TQDM_COLOURS[pol_id % len(TQDM_COLOURS)]
        tic = time.perf_counter()
        for day in tqdm(range(1, Ndays+1), desc=desc, colour=colour):
            tour = []
            if bins.is_stochastic():
                new_overflows, fill, total_fill, sum_lost = bins.stochasticFilling()
            else:
                new_overflows, fill, total_fill, sum_lost = bins.loadFilling(day)
            overflows += new_overflows
            if 'policy_last_minute_and_path' in policy:
                last_minute_cf = int(policy.rsplit("_and_path", 1)[1])
                if last_minute_cf not in [50, 70, 90]:
                    print('Valid cf values for policy_last_minute_and_path: [50, 70, 90]')
                    raise ValueError(f'Invalid cf value for policy_last_minute_and_path: {last_minute_cf}')
                bins.setCollectionLvlandFreq(cf=last_minute_cf/100)
                tour = policy_last_minute_and_path(bins.c, distancesC, pathbetweenstates, bins.collectlevl, waste_type, area)
                cost = get_route_cost(distance_matrix, tour) if tour else 0
            elif 'policy_last_minute' in policy:
                last_minute_cf = int(policy.rsplit("_last_minute", 1)[1])
                if last_minute_cf not in [50, 70, 90]:
                    print('Valid cf values for policy_last_minute: [50, 70, 90]')
                    raise ValueError(f'Invalid cf value for policy_last_minute: {last_minute_cf}')
                bins.setCollectionLvlandFreq(cf=last_minute_cf/100)
                tour = policy_last_minute(bins.c, distancesC, bins.collectlevl, waste_type, area)
                cost = get_route_cost(distance_matrix, tour) if tour else 0
            elif 'policy_regular' in policy:
                regular_level = int(policy.rsplit("_regular", 1)[1]) - 1
                if regular_level not in [1, 2, 5]:
                    print('Valid lvl values for policy_regular: [2, 3, 6]')
                    raise ValueError(f'Invalid lvl value for policy_regular: {regular_level + 1}')
                tour = policy_regular(bins.n, bins.c, distancesC, regular_level, day, cached, waste_type, area)
                cost = get_route_cost(distance_matrix, tour) if tour else 0
                if cached is not None and not cached and tour: cached = tour
            elif policy[:2] == 'am' or policy[:4] == 'ddam' or "transgcn" in policy:
                daily_data = set_daily_waste(model_data, bins.c, device, fill)
                tour, cost, output_dict = model.compute_simulator_day(
                    daily_data, graph, distC_tensor, profit_vars, run_tsp, hrl_manager=hrl_manager, waste_history=bins.get_fill_history(device=device)
                )
                attention_dict[policy].append(output_dict)
            elif 'gurobi' in policy:
                gp_param = float(policy.rsplit("_vrpp", 1)[1])
                try:
                    to_collect = policy_gurobi_vrpp(bins.c, dist_matrix_edges.tolist(), gp_env, gp_param, 
                                                    bins.means, bins.std, waste_type, area, time_limit=600)
                except:
                    to_collect = policy_gurobi_vrpp(bins.c, dist_matrix_edges.tolist(), gp_env, gp_param, 
                                                    bins.means, bins.std, waste_type, area, time_limit=3600)

                if to_collect:
                    tour = find_route(distancesC, np.array(to_collect[0])) if run_tsp else to_collect[0]
                    cost = get_route_cost(dist_matrix_edges, tour)
            elif 'hexaly' in policy:
                hex_param = float(policy.rsplit("_vrpp", 1)[1])
                try:
                    routes = policy_hexaly_vrpp(bins.c, distance_matrix.tolist(), hex_param, bins.means, bins.std, waste_type, area, time_limit=600)
                except:
                    routes = policy_hexaly_vrpp(bins.c, distance_matrix.tolist(), hex_param, bins.means, bins.std, waste_type, area, time_limit=3600)
                
                if routes:
                    tour = find_route(distancesC, np.array(routes[0])) if run_tsp else routes[0]
                    cost = get_route_cost(dist_matrix_edges, tour)
            elif 'policy_look_ahead' in policy:
                look_ahead_config = policy[policy.find('ahead_') + len('ahead_')]
                try:
                    chosen_combination = look_ahead_configurations[look_ahead_config]
                except KeyError as ke:
                    print('Possible policy_look_ahead configurations:')
                    for pos_pol, ploa_configs in look_ahead_configurations.items():
                        print(f'{pos_pol} configuration: {ploa_configs}')
                    raise ValueError(f'Invalid policy_look_ahead configuration: {policy}')

                binsids = np.arange(0, number_of_bins).tolist()
                must_go_bins = policy_lookahead(binsids, bins.c, bins.means, current_collection_day)
                if len(must_go_bins) > 0:
                    vehicle_capacity, R, B, C, E = load_area_and_waste_type_params(area, waste_type)
                    values = {
                        'R': R, 'C': C, 'E': E, 'B': B, 
                        'vehicle_capacity': vehicle_capacity,
                    }
                    if 'vrpp' in policy:
                        values['time_limit'] = 600
                        fh = np.array(fill_history).transpose()
                        routes, profit, _ = policy_lookahead_vrpp(fh, binsids, must_go_bins, distance_matrix, values, env=gp_env)
                        if routes:
                            tour = find_route(distancesC, np.array(routes)) if run_tsp else routes
                            cost = get_route_cost(distance_matrix, tour)
                    elif 'sans' in policy:
                        values['time_limit'] = 60
                        fh = np.array(fill_history).transpose()
                        T_min = 0.01
                        T_init = 75
                        iterations_per_T = 50000
                        alpha = 0.7
                        params = (T_init, iterations_per_T, alpha, T_min)
                        routes, profit, _ = policy_lookahead_sans(fh, coordinates, distance_matrix, params, must_go_bins, values, binsids)
                        if routes:
                            tour = find_route(distancesC, np.array(routes[0])) if run_tsp else routes[0]
                            cost = get_route_cost(distance_matrix, tour)
                    else:
                        values['shift_duration'] = 390 # minutes
                        values['perc_bins_can_overflow'] = 0 # 0%
                        points = create_points(new_data, coordinates)
                        new_data.loc[1:number_of_bins+1, 'Stock'] = (bins.c/100).astype('float32')
                        new_data.loc[1:number_of_bins+1, 'Accum_Rate'] = (bins.means/100).astype('float32')
                        try:
                            routes, profit, removed_bins = find_solutions(new_data, coordinates, distance_matrix, chosen_combination,
                                                                        must_go_bins, values, number_of_bins, points, time_limit=600)
                        except:
                            routes, profit, removed_bins = find_solutions(new_data, coordinates, distance_matrix, chosen_combination,
                                                                        must_go_bins, values, number_of_bins, points, time_limit=3600)
                        
                        if routes:
                            tour = find_route(distancesC, np.array(routes[0])) if run_tsp else routes[0]
                            cost = get_route_cost(distance_matrix, tour)
                else:
                    tour = [0, 0]
                    cost = 0
            else:
                raise ValueError("Unknown policy:", policy)
            cost_ls.append(cost)
            tour_ls.append(tour)
            print("Tour (cost {}): {}".format(cost, tour))
            collected, total_collected, ncol, profit = bins.collect(tour, cost)
            dlog = get_daily_results(total_collected, ncol, cost, tour, day, new_overflows, sum_lost, coordinates, profit)
            for key, val in dlog.items():
                daily_log[key].append(val)
        full_daily_log["{}#{}".format(pol, sample_id)] = daily_log
        lg = [np.sum(bins.inoverflow), np.sum(bins.collected), np.sum(bins.ncollections), 
          np.sum(bins.lost), bins.travel, np.nan_to_num(np.sum(bins.collected)/bins.travel, 0), 
          np.sum(bins.inoverflow)-np.sum(bins.collected)+bins.travel, bins.ndays, time.perf_counter()-tic]
        if Nsamples > 1:
            save_id = sample_id - start_id
            tmp_log[pol].append(lg)
            log_to_json(logfull_filepath, SIM_METRICS, {pol: tmp_log[pol][save_id]}, sample_id=sample_id)
            log_to_json(daily_log_path, DAY_METRICS, {f"{pol} #{sample_id}": daily_log.values()})
        else:
            log.append(lg)
            log_to_json(log_filepath, SIM_METRICS, {pol: log[-1]})
            log_to_json(daily_log_path, DAY_METRICS, {pol: daily_log.values()})

amgat_hrl_gamma1 #0:   7%|▋         | 2/30 [00:00<00:01, 14.05it/s]

Tour (cost 3238): [0, 59, 57, 1, 97, 89, 5, 85, 58, 100, 75, 71, 39, 88, 63, 47, 67, 25, 27, 29, 79, 69, 17, 93, 9, 53, 33, 86, 49, 77, 19, 11, 41, 60, 66, 35, 65, 37, 73, 99, 10, 91, 68, 30, 87, 13, 90, 7, 96, 95, 64, 48, 28, 16, 31, 51, 98, 45, 78, 50, 20, 81, 43, 26, 6, 21, 92, 56, 23, 38, 36, 3, 8, 61, 18, 40, 44, 54, 76, 70, 34, 46, 55, 22, 62, 42, 80, 14, 82, 12, 84, 32, 74, 52, 15, 4, 94, 83, 72, 2, 24, 0]
Tour (cost 3195): [0, 5, 25, 89, 97, 57, 59, 1, 58, 49, 63, 85, 88, 71, 77, 9, 100, 27, 39, 29, 67, 37, 75, 86, 13, 11, 47, 19, 99, 68, 60, 66, 69, 17, 10, 78, 23, 65, 64, 45, 35, 91, 90, 7, 70, 16, 30, 79, 50, 43, 48, 95, 36, 21, 40, 33, 28, 56, 20, 44, 61, 92, 93, 55, 98, 41, 54, 26, 62, 3, 87, 96, 18, 76, 51, 6, 38, 82, 8, 22, 32, 81, 42, 80, 14, 52, 84, 4, 31, 73, 12, 53, 46, 74, 34, 94, 15, 83, 24, 72, 2, 0]
Tour (cost 3043): [0, 57, 59, 97, 89, 58, 88, 5, 1, 85, 67, 71, 60, 75, 100, 63, 37, 47, 49, 7, 69, 25, 9, 86, 61, 77, 39, 27, 79, 66, 33, 11, 65, 90, 68, 19, 30, 99,

amgat_hrl_gamma1 #0:  20%|██        | 6/30 [00:00<00:01, 13.86it/s]

Tour (cost 3400): [0, 57, 5, 59, 71, 47, 58, 89, 97, 1, 67, 88, 25, 75, 19, 39, 100, 63, 85, 86, 69, 37, 27, 60, 7, 49, 66, 9, 79, 33, 61, 90, 95, 65, 17, 77, 45, 78, 81, 48, 16, 68, 93, 35, 23, 50, 20, 13, 11, 3, 30, 6, 87, 99, 29, 36, 64, 18, 91, 40, 98, 8, 43, 53, 21, 10, 92, 28, 44, 70, 62, 26, 56, 38, 31, 51, 41, 54, 82, 73, 80, 12, 22, 14, 46, 42, 55, 84, 76, 32, 52, 34, 4, 96, 74, 94, 83, 2, 72, 24, 15, 0]
Tour (cost 3363): [0, 59, 5, 89, 85, 57, 47, 58, 1, 88, 97, 25, 75, 69, 63, 17, 71, 7, 19, 100, 49, 86, 37, 13, 60, 9, 39, 27, 73, 11, 33, 29, 77, 90, 66, 68, 55, 50, 65, 79, 30, 41, 10, 31, 36, 67, 48, 6, 99, 91, 93, 53, 70, 78, 23, 21, 92, 35, 3, 45, 64, 8, 16, 26, 95, 43, 76, 81, 87, 98, 61, 20, 56, 18, 54, 44, 96, 28, 62, 40, 22, 14, 38, 12, 82, 32, 80, 51, 46, 42, 52, 34, 83, 84, 4, 74, 94, 15, 2, 24, 72, 0]
Tour (cost 3154): [0, 59, 89, 37, 57, 97, 5, 1, 58, 47, 19, 88, 63, 100, 71, 25, 85, 77, 86, 60, 75, 99, 29, 66, 69, 33, 79, 7, 67, 17, 87, 49, 13, 90, 65, 31, 39, 21

amgat_hrl_gamma1 #0:  27%|██▋       | 8/30 [00:00<00:01, 14.01it/s]

...

amgat_hrl_gamma1 #1:  67%|██████▋   | 20/30 [00:01<00:00, 12.15it/s]

Tour (cost 3301): [0, 59, 97, 57, 1, 89, 58, 47, 5, 7, 67, 88, 75, 100, 85, 25, 71, 87, 63, 9, 49, 60, 86, 39, 19, 69, 66, 65, 13, 91, 37, 29, 56, 99, 95, 77, 90, 81, 27, 98, 78, 45, 93, 50, 79, 17, 10, 61, 35, 48, 26, 64, 68, 33, 28, 23, 30, 62, 16, 11, 43, 80, 92, 36, 70, 6, 55, 38, 8, 51, 18, 44, 96, 73, 40, 32, 54, 46, 20, 31, 21, 14, 22, 3, 52, 53, 34, 41, 42, 76, 82, 12, 83, 74, 84, 4, 94, 72, 2, 24, 15, 0]
Tour (cost 3244): [0, 59, 5, 89, 97, 25, 1, 85, 57, 58, 88, 71, 29, 37, 67, 47, 63, 87, 75, 9, 100, 86, 53, 91, 60, 19, 43, 65, 69, 66, 95, 78, 10, 27, 17, 99, 45, 77, 11, 7, 23, 39, 49, 36, 13, 50, 68, 90, 16, 93, 26, 21, 79, 30, 35, 48, 64, 40, 33, 73, 31, 18, 56, 28, 98, 81, 84, 6, 22, 20, 92, 54, 38, 44, 70, 8, 76, 51, 80, 3, 62, 82, 14, 41, 32, 42, 46, 12, 52, 61, 55, 34, 96, 4, 74, 94, 15, 83, 2, 24, 72, 0]
Tour (cost 3345): [0, 89, 59, 85, 57, 5, 71, 1, 58, 97, 39, 100, 88, 63, 67, 87, 69, 27, 47, 86, 9, 75, 25, 11, 99, 29, 65, 49, 77, 66, 60, 90, 73, 16, 13, 79, 7, 68,

amgat_hrl_gamma1 #1:  73%|███████▎  | 22/30 [00:01<00:00, 12.40it/s]


Tour (cost 3176): [0, 89, 59, 57, 1, 5, 69, 97, 58, 25, 88, 100, 91, 77, 17, 29, 71, 75, 47, 37, 85, 63, 60, 99, 49, 67, 87, 86, 39, 33, 78, 65, 19, 20, 66, 9, 68, 79, 6, 7, 90, 16, 13, 80, 64, 30, 48, 40, 45, 27, 23, 11, 70, 56, 95, 36, 81, 50, 98, 35, 61, 53, 92, 38, 8, 44, 10, 18, 15, 93, 26, 41, 54, 28, 73, 14, 62, 21, 46, 22, 76, 55, 96, 52, 43, 32, 31, 42, 3, 82, 84, 12, 51, 34, 74, 4, 94, 83, 24, 2, 72, 0]


KeyboardInterrupt: 

## To-Do
### Comecar escrever o paper

In [13]:
import json
import statistics

if Nsamples > 1:
    log_mean = []
    log_std = []
    for pol_log in tmp_log.values():
        log_mean.append([*map(statistics.mean, zip(*pol_log))])
        log_std.append([*map(statistics.stdev, zip(*pol_log))])

    for lg, pol in zip(log_mean, policies):
        print(f'"{pol}":', end=" ")   
        print(json.dumps(dict(zip(SIM_METRICS, lg)), indent=True))

    print("Standard deviation")
    for lg, pol in zip(log_std, policies):
        print(f'"{pol}":', end=" ")
        print(json.dumps(dict(zip(SIM_METRICS, lg)), indent=True))
else:
    for lg, pol in zip(log, policies):
        print(f'"{pol}":', end=" ")
        print(json.dumps(dict(zip(SIM_METRICS, lg)), indent=True))

"amgat_hrl_gamma1": {
 "overflows": 12.8,
 "kg": 37507.52712744805,
 "ncol": 3000.0,
 "kg_lost": 90.15397004821467,
 "km": 97476.9,
 "kg/km": 0.3847889282769488,
 "cost": 59982.17287255196,
 "profit": 30,
 "days": 2.422922183398623
}
Standard deviation
"amgat_hrl_gamma1": {
 "overflows": 2.5298221281347035,
 "kg": 222.36043991586314,
 "ncol": 0.0,
 "kg_lost": 18.04267606063271,
 "km": 313.4813302964699,
 "kg/km": 0.0028468520126740744,
 "cost": 425.35007188996946,
 "profit": 0.0,
 "days": 0.2774236467614045
}


In [ ]:
samples_attn_viz = [0] # list(range(0, Nsamples))

for name in model_names:
    for sample_idx in samples_attn_viz:
        for layer_idx in range(configs[name]['n_encode_layers']):
            for head_idx in range(configs[name]['n_heads']):
                indices = indices_ls[sample_id]
                labels = ['Depot'] + list(map(lambda id: "Bin {}".format(id), data.iloc[indices]['ID'].tolist()))
                attn_maps = plot_attention_maps_wrapper(home_dir, Ndays, number_of_bins, "output", area, attention_dict, name, 
                                                        log_plot, layer_idx, sample_idx, head_idx, x_labels=labels, y_labels=labels)

In [ ]:
metrics_viz = ['cost']
policies_daily_viz = policies[:-1]
days_ls = [x for x in range(1, Ndays+1)]

plt.figure(dpi=200)
for metric in metrics_viz:
    plt.title(f"Daily {metric} (Mean with Min/Max range over all simulation samples)")
    plt.xlabel("Day")
    plt.ylabel(f"{metric.capitalize()}")
    for viz_pol in policies_daily_viz:
        metric_pol_data = []
        for sample_id in range(Nsamples):
            metric_pol_data.append(full_daily_log[f"{viz_pol}#{sample_id}"][metric])

        metric_arr = np.array(metric_pol_data)
        means = np.mean(metric_arr, axis=0)
        mins = np.min(metric_arr, axis=0)
        maxs = np.max(metric_arr, axis=0)
        plt.plot(days_ls, means, marker='o', linestyle='-', label=viz_pol)
        plt.fill_between(days_ls, mins, maxs, alpha=0.2)
    
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

### Save Logs

In [ ]:
dit = {}
for pol, val in zip(policies, log_mean):
    dit[pol] = val if isinstance(val, list) else val.tolist()

log_filepath = os.path.join(output_dir, f'log_mean_{Nsamples}N.json')
log_to_json(log_filepath, SIM_METRICS, dit)
if IN_COLAB:
    log_to_pickle(os.path.join(output_dir, f'log_mean_{Nsamples}N.pkl'), log_mean, dw_func=gfiles.download)

if Nsamples > 1:
    std_dit = {}
    for pol, val in zip(policies, log_std):
        std_dit[pol] = val if isinstance(val, list) else val.tolist()
    
    logstd_filepath = os.path.join(output_dir, f'log_std_{Nsamples}N.json')
    log_to_json(logstd_filepath, SIM_METRICS, std_dit)
    if IN_COLAB:
        log_to_pickle(os.path.join(output_dir, f'log_std_{Nsamples}N.pkl'), log_std, dw_func=gfiles.download)